In [14]:
import cv2
import os
import pandas as pd
import numpy as np
import rasterio
from rasterio import mask as raster_mask
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely
from shapely import speedups

speedups.disable()



In [2]:
import utils

In [3]:
from skimage.transform import resize

In [4]:
grid_sizes = utils.get_grid_sizes()


# Structure

"train" folder will contain one numpy mask for each classtype for every train image (train_y) and an adjusted numpy array for every 16 band raster file (train_X).


In [16]:
train_X_path = os.path.join('train','train_X')
train_y_path = os.path.join('train','train_y')
train_path = 'train'

train_dirs = [train_path,train_X_path,train_y_path]

for directory in train_dirs:
    if not os.path.exists(directory):
        os.mkdir(directory)

                      

# 1: Prepare and scale train polygons

In [8]:
### this will be the standard image size for training & testing
base_size = 835

In [18]:
wkt_train = pd.read_csv('train_wkt_v4.csv')
wkt_train['MultipolygonWKT'] = wkt_train['MultipolygonWKT'].apply(shapely.wkt.loads)

train_gpd = gpd.GeoDataFrame(wkt_train,geometry='MultipolygonWKT')
train_gpd.rename({'MultipolygonWKT':'geometry'},axis=1,inplace=True)

In [19]:
def get_src_h_w(src):
    w = src.height
    h = src.width

    return w, h

In [20]:
def get_H_W_scalers(w,H):
    W = w**2 / (w+1)
    H = h**2 / (h+1)
    return W,H

In [21]:
# def get_mask(w,h,exterior,interior,classtype):
#     img_mask = np.zeros((w,h), np.uint8)
#     cv2.fillPoly((w,h), exterior, classtype)
#     if len(interior) > 0:
#         cv2.fillPoly((w,h), interior, 0)
#     return img_mask


In [22]:
train_gpd['dummy_geo'] = 0
train_gpd['dummy_geo'] = train_gpd['dummy_geo'].astype('object')
train_ids = train_gpd['ImageId'].unique()

raster_dict = {}
img_shape = (base_size,base_size)

In [12]:
for image_id in train_ids:
    img = rasterio.open(os.path.join('three_band',image_id+'.tif'))
    w, h = get_src_h_w(img)
#     w, h = img_shape
    W, H = get_H_W_scalers(h,w)
    # store w, h in a dictionary for sixteen band rasters
    
    raster_dict[image_id] = (w,h)
    
    xmax, ymax = utils.get_image_max(image_id, grid_sizes)
    
    for index, row in train_gpd.loc[train_gpd['ImageId']==image_id].iterrows():
        classtype = row['ClassType']
        geo = row['geometry']
        if str(geo) == "GEOMETRY COLLECTION EMPTY" or str(geo) == 'MULTIPOLYGON EMPTY':
            train_gpd.at[index,'dummy_geo'] = np.nan
            mask = np.zeros((img_shape),np.uint8)

        else:
            ### scale polygons
            polys = []
            for poly in geo:

                x_ext,y_ext = np.array(poly.exterior.coords.xy[0]), np.array(poly.exterior.coords.xy[1])
                exterior = utils.convert_xy_to_raster(x_ext,y_ext,xmax,ymax,W,H)
                interiors = []
                for interior in poly.interiors:
                    x_int, y_int = np.array(interior.coords.xy[0]),np.array(interior.coords.xy[1])
                    interiors.append(utils.convert_xy_to_raster(x_int, y_int,xmax,ymax,W,H))

                scaled_poly = shapely.geometry.Polygon(exterior,interiors)

                polys.append(scaled_poly)

            polygon_val = shapely.geometry.MultiPolygon(polys)
            train_gpd.at[index,'dummy_geo'] = polygon_val
            ### get polygon mask
#             print(image_id,classtype)
#             mask = get_mask(w,h,exterior,interiors,classtype)
            mask = raster_mask.mask(img,polygon_val,all_touched=True)[0][0]
            resized_mask = resize(mask,img_shape,anti_aliasing=False)
            mask = np.where(resized_mask==np.min(resized_mask),0,1)
        
        ### save mask
        classtype = row['ClassType']
        np.save(os.path.join(train_y_path,image_id+"_"+str(classtype)),mask)
    img.close()
    


C:\ProgramData\Miniconda3\envs\gis_test\lib\site-packages\rasterio\__init__.py:221: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


TypeError: can only concatenate str (not "int") to str

In [ ]:
train_gpd.set_geometry('dummy_geo',inplace=True)
train_gpd.drop('geometry',axis=1,inplace=True)
train_gpd.to_file('train_gdf_'+str(img_shape[0])+'.geojson',driver=GeoJSON)

# 2: Create X_train, X_label

X_train will be the combined 16 - band images.
X_label will be a mask of the polygons

In [13]:
# raster dict created in previous step to get the h and w of the thirteen band images. Keys = image id, values = (w,h) tuple

band_suffix = ["P","M","A"]

Based on the tags in the sixteen band rasters, we know that P is panchromatic (band 1), M is multispectral (bands 2 - 9), and A is SWIR (bands 10-17). We'll read each image into a numpy array and rename the bands accordingly.

In [14]:
for image_id in train_ids:
#     w,h = raster_dict[image_id]
    array_list = []
    for band in band_suffix:
        img = rasterio.open(os.path.join('sixteen_band',image_id+"_"+band+".tif"))
        scaled_img = img.read(out_shape=(img_shape[0],img_shape[1]))
        array_list.append(scaled_img)
        img.close()
    train_X = np.concatenate(array_list)
    np.save(os.path.join(train_X_path,image_id),train_X)
        

In [15]:
# remove train dataframe to save memory
train_gpd=None

In [19]:
x_tr = np.load(os.path.join(train_X_path,train_ids[0]+".npy"))

In [20]:
x_tr.shape

(17, 835, 835)

In [23]:
y_tr = np.load(os.path.join(train_y_path,train_ids[0]+"_1.npy"))

In [24]:
y_tr.shape

(835, 835)

In [29]:
# Split into training and test sets

train_imgs = train_ids[:20]
test_imgs = train_ids[20:]

In [ ]:
data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True,
                     rotation_range=90,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2)
image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)
mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)

In [30]:
import tensorflow as tf

# Model Test: Class 1


In [1]:
# from keras.models import Model, load_model
# from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout
# from keras.optimizers import Adam
# from keras.callbacks import ModelCheckpoint, LearningRateScheduler
# from keras import backend as K

In [2]:
# smooth = 1e-12

In [10]:
# def jaccard_coef(y_true, y_pred):
#     # __author__ = Vladimir Iglovikov
#     intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
#     sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

#     jac = (intersection + smooth) / (sum_ - intersection + smooth)

#     return K.mean(jac)


# def jaccard_coef_int(y_true, y_pred):
#     # __author__ = Vladimir Iglovikov
#     y_pred_pos = K.round(K.clip(y_pred, 0, 1))

#     intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
#     sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])
#     jac = (intersection + smooth) / (sum_ - intersection + smooth)
#     return K.mean(jac)

# def get_unet():
#     inputs = Input((17, base_size, base_size))
#     conv1 = Convolution2D(32, 3, 3, activation='relu')(inputs)
#     conv1 = Convolution2D(32, 3, 3, activation='relu')(conv1)
#     pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

#     conv2 = Convolution2D(64, 3, 3, activation='relu')(pool1)
#     conv2 = Convolution2D(64, 3, 3, activation='relu')(conv2)
#     pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

#     conv3 = Convolution2D(128, 3, 3, activation='relu')(pool2)
#     conv3 = Convolution2D(128, 3, 3, activation='relu')(conv3)
#     pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

#     conv4 = Convolution2D(256, 3, 3, activation='relu')(pool3)
#     conv4 = Convolution2D(256, 3, 3, activation='relu')(conv4)
#     pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

#     conv5 = Convolution2D(512, 3, 3, activation='relu')(pool4)
#     conv5 = Convolution2D(512, 3, 3, activation='relu')(conv5)

#     up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1)
#     conv6 = Convolution2D(256, 3, 3, activation='relu')(up6)
#     conv6 = Convolution2D(256, 3, 3, activation='relu')(conv6)

#     up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=1)
#     conv7 = Convolution2D(128, 3, 3, activation='relu')(up7)
#     conv7 = Convolution2D(128, 3, 3, activation='relu')(conv7)

#     up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)
#     conv8 = Convolution2D(64, 3, 3, activation='relu')(up8)
#     conv8 = Convolution2D(64, 3, 3, activation='relu')(conv8)

#     up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)
#     conv9 = Convolution2D(32, 3, 3, activation='relu')(up9)
#     conv9 = Convolution2D(32, 3, 3, activation='relu')(conv9)

#     conv10 = Convolution2D(N_Cls, 1, 1, activation='sigmoid')(conv9)

#     model = Model(input=inputs, output=conv10)
#     model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[jaccard_coef, jaccard_coef_int, 'accuracy'])
#     return model

In [35]:
def load_train(classtype,train_ids,batch_size=5):
    while True:
        batch = np.random.choice(train_ids, size = batch_size)
        X_arr = []
        y_arr = []
        for train_id in batch:
            X = np.load(os.path.join(train_X_path,train_id+".npy"))
            y = np.load(os.path.join(train_y_path,train_id+"_"+str(classtype)+".npy"))
            
            X_arr.append(X)
            y_arr.append(y)
        
        batch_X = np.array(X_arr)
        batch_y = np.array(y_arr)
        yield (X, y)
                                                                           

In [36]:
train_1_generator = load_train(1,train_imgs)


In [ ]:
# data augmentation

data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True,
                     rotation_range=90,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2)
image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)
mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)



###  alternatively?

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        shear_range=0,
        zoom_range=0,
        rotation_range=90,
        horizontal_flip=True)

img_generator = train_datagen.flow_from_directory(
        's2cloudless_imagery',
        target_size=(512, 512),
        batch_size=len(images),
        class_mode=None, seed=111, shuffle=False)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,    
        shear_range=0,
        zoom_range=0,
        rotation_range=90,
        horizontal_flip=True)

mask_generator = test_datagen.flow_from_directory(
        's2cloudless_label_imagery',
        target_size=(512, 512),
        batch_size=len(images),
        class_mode=None, seed=111, shuffle=False)

In [6]:
def train_net(classtype):
    print("start train net")

#     x_trn, y_trn = get_patches(img, msk)

    model = get_unet()
#     model.load_weights('weights/unet_10_jk0.7878')
#     model_checkpoint = ModelCheckpoint('weights/unet_tmp.hdf5', monitor='loss', save_best_only=True)
    for i in range(1):
        model.fit_generator(load_train(classtype), batch_size=64, nb_epoch=1, verbose=1, shuffle=True,
                  #callbacks=[model_checkpoint], 
                  validation_data=(x_val, y_val))


        score, trs = calc_jacc(model)
        print('val jk', score)

    return model


In [11]:
train_net(1)

start train net


ValueError: Negative dimension size caused by subtracting 2 from 1 for '{{node max_pooling2d/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1]](conv2d_1/Identity)' with input shapes: [?,1,92,32].

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pandas as pd
from shapely.wkt import loads as wkt_loads
import tifffile as tiff
import os
import random
from keras.models import Model, load_model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from sklearn.metrics import jaccard_similarity_score
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
from collections import defaultdict


In [ ]:
def get_train_class_polygons(train_df, imageId, class_type):
    df_image = wkt_list_pandas[wkt_list_pandas.ImageId == imageId]
    multipoly_def = df_image[df_image.ClassType == class_type].MultipolygonWKT
    polygonList = None
    if len(multipoly_def) > 0:
        assert len(multipoly_def) == 1
        polygonList = wkt_loads(multipoly_def.values[0])
    return polygonList


def _get_and_convert_contours(polygonList, raster_img_size, xymax):
    perim_list = []
    interior_list = []
    if polygonList is None:
        return None
    for k in range(len(polygonList)):
        poly = polygonList[k]
        perim = np.array(list(poly.exterior.coords))
        perim_c = _convert_coordinates_to_raster(perim, raster_img_size, xymax)
        perim_list.append(perim_c)
        for pi in poly.interiors:
            interior = np.array(list(pi.coords))
            interior_c = _convert_coordinates_to_raster(interior, raster_img_size, xymax)
            interior_list.append(interior_c)
    return perim_list, interior_list


def _plot_mask_from_contours(raster_img_size, contours, class_value=1):
    img_mask = np.zeros(raster_img_size, np.int8)
    if contours is None:
        return img_mask
    perim_list, interior_list = contours
    cv2.fillPoly(img_mask, perim_list, class_value)
    cv2.fillPoly(img_mask, interior_list, 0)
    return img_mask


def mask_to_polygons_layer(mask):
    all_polygons = []
    for shape, value in features.shapes(mask.astype(np.int16), mask=(mask == 1), transform=rasterio.Affine(1.0, 0, 0, 0, 1.0, 0)):

        all_polygons.append(shapely.geometry.shape(shape))

    all_polygons = shapely.geometry.MultiPolygon(all_polygons)
    if not all_polygons.is_valid:
        all_polygons = all_polygons.buffer(0)
        # Sometimes buffer() converts a simple Multipolygon to just a Polygon,
        # need to keep it a Multi throughout
        if all_polygons.type == 'Polygon':
            all_polygons = shapely.geometry.MultiPolygon([all_polygons])
    return all_polygons


def generate_mask_for_image_and_class(raster_size, imageId, class_type, grid_sizes_panda, wkt_list_pandas):
    xymax = _get_xmax_ymin(grid_sizes_panda, imageId)
    polygon_list = _get_polygon_list(wkt_list_pandas, imageId, class_type)
    contours = _get_and_convert_contours(polygon_list, raster_size, xymax)
    mask = _plot_mask_from_contours(raster_size, contours, 1)
    return mask


def calculate_score_mask(y_true, y_pred):
    """
    Function calculate jaccard score for real vs mask

    :param y_true:
    :param y_pred:
    :return:
    """
    num_mask_channels = y_true.shape[0]

    result = np.ones(num_mask_channels)

    for mask_channel in range(num_mask_channels):
        intersection = np.dot(y_true[mask_channel, ...].flatten(), y_pred[mask_channel, ...].flatten())
        _sum = y_true[mask_channel, ...].sum() + y_pred[mask_channel, ...].sum()
        if _sum - intersection != 0:
            result[mask_channel] = intersection / (_sum - intersection)
    return np.mean(result)


def mask2polygons(mask, image_id):
    """

    :param mask:
    :return: list of the type: [Polygons_class_1, Polygons_class_2]
    """
    W = mask.shape[1]
    H = mask.shape[2]

    num_mask_channels = mask.shape[0]

    x_max = gs.loc[gs['ImageId'] == image_id, 'Xmax'].values[0]
    y_min = gs.loc[gs['ImageId'] == image_id, 'Ymin'].values[0]

    W_ = W * (W / (W + 1))
    H_ = H * (H / (H + 1))

    x_scaler = W_ / x_max
    y_scaler = H_ / y_min

    x_scaler = 1 / x_scaler
    y_scaler = 1 / y_scaler

    result = []

    for mask_channel in range(num_mask_channels):
        polygons = mask_to_polygons_layer(mask[mask_channel])

        polygons = shapely.affinity.scale(polygons, xfact=x_scaler, yfact=y_scaler, origin=(0, 0, 0))

        if not polygons.is_valid:
            polygons = polygons.buffer(0)
            # Sometimes buffer() converts a simple Multipolygon to just a Polygon,
            # need to keep it a Multi throughout
            polygons = shapely.geometry.MultiPolygon([polygons])

        result += [str(polygons)]

    return result


def polygons2mask(polygons, width, height, image_id):
    xymax = _get_xmax_ymin(gs, image_id)

    mask = np.zeros((len(polygons), width, height))

    for i, p in enumerate(polygons):
        polygon_list = wkt_loads(p)
        if polygon_list.length == 0:
            continue
        contours = _get_and_convert_contours(polygon_list, (width, height), xymax)
        mask[i] = _plot_mask_from_contours((width, height), contours, 1)
    return mask


def generate_mask(image_id, width, height, num_mask_channels=10):
    mask = np.zeros((num_mask_channels, width, height))

    for mask_channel in range(num_mask_channels):
        mask[mask_channel, :, :] = generate_mask_for_image_and_class((width, height), image_id, mask_channel + 1, gs, train_wkt)
    return mask


def calculate_polygon_match(image_id, num_mask_channels=10):
    """
    calculates jaccard index between before poly and after poly

    Ideally should be 1

    :param image_id:
    :param num_mask_channels:
    :return:
    """

    image = tiff.imread(os.path.join(data_path, 'three_band', image_id + '.tif'))

    width = image.shape[1]
    height = image.shape[2]

    mask_before = generate_mask(image_id, width, height, num_mask_channels=num_mask_channels)

    polygons = mask2polygons(mask_before, image_id)
    predicted_mask = polygons2mask(polygons, width, height, image_id)

    mask = generate_mask(image_id, width, height)

    return calculate_score_mask(mask, predicted_mask)


if __name__ == '__main__':
    for image_id in train_wkt['ImageId'].unique():
        print(image_id, calculate_polygon_match(image_id))

In [1]:
from keras.datasets import mnist

# use Keras to import pre-shuffled MNIST database
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [12]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)